In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

df = pd.read_csv('pix.csv', sep=';')

print("Primeiras 5 linhas do dataset:")
print(df.head())
print("\nInformações do dataset:")
print(df.info())
print("\nEstatísticas descritivas:")
print(df.describe())

print("\nValores nulos por coluna:")
print(df.isnull().sum())

df['Valor'].fillna(df['Valor'].mean(), inplace=True)

categorical_columns = ['Moeda', 'Pagador_Nome', 'Pagador_CPF_CNPJ', 'Pagador_Banco', 
                      'Recebedor_Nome', 'Recebedor_CPF_CNPJ', 'Recebedor_Banco', 
                      'ChavePix_Utilizada', 'TipoChave', 'Descricao', 'Status']
for column in categorical_columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

dummy_columns = ['Moeda', 'Pagador_Banco', 'Recebedor_Banco', 'TipoChave', 'Status']
df = pd.get_dummies(df, columns=dummy_columns, drop_first=True)

X = df.drop(['Anomalia', 'EndToEndId', 'DataHora', 'Pagador_Nome', 
             'Pagador_CPF_CNPJ', 'Recebedor_Nome', 'Recebedor_CPF_CNPJ', 
             'ChavePix_Utilizada', 'Descricao'], axis=1)
y = df['Anomalia']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nTamanho do conjunto de treino:", X_train.shape)
print("Tamanho do conjunto de teste:", X_test.shape)

models = {
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'KNeighbors': KNeighborsClassifier(),
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000)
}

results = {'Model': [], 'Accuracy': [], 'F1-Score': []}

for name, model in models.items():
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    results['Model'].append(name)
    results['Accuracy'].append(accuracy)
    results['F1-Score'].append(f1)
    
    print(f"\nResultados para {name}:")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"Matriz de Confusão:\n{cm}")

plt.figure(figsize=(8, 5))
plt.bar(results['Model'], results['Accuracy'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.title('Comparação de Acurácia dos Modelos')
plt.xlabel('Modelo')
plt.ylabel('Acurácia')
plt.ylim(0, 1)
for i, v in enumerate(results['Accuracy']):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center')
plt.tight_layout()
plt.show()